# Data Prep+Aug

In [1]:
import pandas as pd
import numpy as np
import os
import csv
import cv2

In [2]:
driving_log = pd.DataFrame.from_csv('sample_data/driving_log.csv')
driving_log.describe()

,steering,throttle,brake,speed
count,8036.000000,8036.000000,8036.000000,8036.000000
mean,0.004070,0.869660,0.001970,28.169839
std,0.128840,0.301326,0.036565,6.149327
min,-0.942695,0.000000,0.000000,0.502490
25%,0.000000,0.985533,0.000000,30.183093
50%,0.000000,0.985533,0.000000,30.186400
75%,0.000000,0.985533,0.000000,30.186640
max,1.000000,0.985533,1.000000,30.709360


In [ ]:
# initial loop, no augmentation

"""
for line in lines[1:]:
    fn = line[0] #.split('/')[-1]
    img = cv2.imread('sample_data/'+fn)
    images.append(img)
    measurements.append(float(line[3]))    
"""
print()

In [ ]:
# second loopo, added flipping

"""
for line in lines[1:]:
    fn = line[0] #.split('/')[-1]
    img = cv2.imread('sample_data/'+fn)
    
    images.append(img)
    measurements.append(float(line[3]))    
        
    images.append(cv2.flip(img,1))
    measurements.append( -1.0 * float(line[3])) 
"""
print()

In [3]:
# third loop, added multiple cameras

CORRECTION = 0.2

images = []
measurements = []

with open('sample_data/driving_log.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader, None)
    c = 0
    for row in reader:
        img_center = cv2.imread('sample_data/' + row[0].strip())
        img_left = cv2.imread('sample_data/' + row[1].strip())
        img_right = cv2.imread('sample_data/' + row[2].strip())
        img = [img_center, img_left, img_right]
        
        steer_center = float(row[3])
        steer_left = steer_center + CORRECTION
        steer_right = steer_center - CORRECTION
        steer = [steer_center, steer_left, steer_right]

        # normal
        images.extend(img)
        # images.append(img_center)
        measurements.extend(steer)    

        # flipped
        images.extend([cv2.flip(i,1) for i in img])
        measurements.extend([-1.0 * s for s in steer])   
        
        if c > 5000: break
        c += 1

In [4]:
X_train = np.array(images)
y_train = np.array(measurements)

In [5]:
X_train.shape

(30012, 160, 320, 3)

---
# Model Architecture

In [7]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers import Cropping2D

#from keras.layers.pooling

In [8]:
EPOCHS = 10

In [9]:
model = Sequential()

# normalize
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160, 320, 3)))

# crop
model.add(Cropping2D(cropping=((70,25), (0,0))))

In [ ]:
# dummy architecture
"""
model.add(Flatten())
model.add(Dense(1))
"""

In [10]:
# nvidia architecture

model.add(Convolution2D(nb_filter=24,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(nb_filter=36,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))
model.add(Convolution2D(nb_filter=48,  nb_row=5, nb_col=5, subsample=(2,2), activation='relu'))

model.add(Convolution2D(nb_filter=64,  nb_row=3, nb_col=3, activation='relu'))
model.add(Convolution2D(nb_filter=64,  nb_row=3, nb_col=3, activation='relu'))

model.add(Flatten())

model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(1))

In [ ]:
model.compile(loss='mse', optimizer='adam')

model.fit(X_train, y_train, validation_split=0.2, shuffle=True, nb_epoch=EPOCHS, verbose=1)

Train on 24009 samples, validate on 6003 samples
Epoch 1/10
24009/24009 [==============================] - 55s - loss: 0.0173 - val_loss: 0.0274
Epoch 2/10
24009/24009 [==============================] - 51s - loss: 0.0136 - val_loss: 0.0294
Epoch 3/10
24009/24009 [==============================] - 51s - loss: 0.0125 - val_loss: 0.0270
Epoch 4/10
24009/24009 [==============================] - 51s - loss: 0.0114 - val_loss: 0.0274
Epoch 5/10
24009/24009 [==============================] - 51s - loss: 0.0107 - val_loss: 0.0297
Epoch 6/10
 6720/24009 [=======>......................] - ETA: 32s - loss: 0.0095

In [ ]:
model.save('models/momo2.h5')